In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/intras-1/images.hdf5
/kaggle/input/intras-1/best_vit_tab.keras
/kaggle/input/intras-1/data_only_numerical.csv
/kaggle/input/intras-1/EFFICIENT_NET.h5
/kaggle/input/intras-1/subject_data.csv
/kaggle/input/intras-1/RESNET_MODEL.h5


In [2]:
data = pd.read_csv("/kaggle/input/intras-1/data_only_numerical.csv")
data.head()

,Unnamed: 0,target,age_approx,sex,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,iddx_full,iddx_1,tbp_lv_dnn_lesion_confidence
0,0,0,60.0,0,3.04,20.244422,16.261975,26.922447,23.954773,33.684638,...,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,0,0,97.517282
1,1,0,60.0,0,1.10,31.712570,25.364740,26.331000,24.549290,41.219030,...,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,0,0,3.141455
2,2,0,60.0,0,3.40,22.575830,17.128170,37.970460,33.485410,44.174920,...,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,0,0,99.804040
3,3,0,65.0,0,3.22,14.242329,12.164757,21.448144,21.121356,25.746200,...,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,0,0,99.989998
4,4,0,55.0,0,2.73,24.725520,20.057470,26.464900,25.710460,36.217980,...,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,0,0,70.442510


In [3]:
data.columns

Index(['Unnamed: 0', 'target', 'age_approx', 'sex', 'clin_size_long_diam_mm',
       'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C',
       'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext',
       'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean',
       'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
       'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM',
       'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
       'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
       'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
       'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'iddx_full', 'iddx_1',
       'tbp_lv_dnn_lesion_confidence'],
      dtype='object')

In [4]:
data = data.drop(['iddx_full','iddx_1','tbp_lv_dnn_lesion_confidence','tbp_lv_nevi_confidence'],axis=1)

In [5]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [6]:
data_1 = pd.read_csv("/kaggle/input/intras-1/subject_data.csv")

/tmp/ipykernel_629/232299192.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data_1 = pd.read_csv("/kaggle/input/intras-1/subject_data.csv")


In [7]:
import h5py
import matplotlib.pyplot as plt
from PIL import Image
import io
import numpy as np

file_path = "/kaggle/input/intras-1/images.hdf5"

# Filter data to get ISIC IDs where the target is 1
isic_codes = data_1[data_1["target"] >= 0][' "isic_id"'].tolist()
images = []
target = data_1['target'].values
with h5py.File(file_path, "r") as hdf:
    ls = list(hdf.keys())  # list all HDF5 keys
    for i in isic_codes:
        image_data = hdf[i]
        # Check if the dataset is scalar and contains bytes
        if image_data.shape == () and image_data.dtype.kind == 'S':
            # Read bytes and open as PIL image
            image_bytes = image_data[()]
            img_pil = Image.open(io.BytesIO(image_bytes))
            images.append(img_pil)
        else:
            print(f"Item '{i}' is not a scalar byte string")

In [8]:
len(target)

401059

In [9]:
data['images_idx'] = [i for i in range(0, len(target))]

In [10]:
mask = data['target'] == 1
df_ones = data[mask].copy()
df_zeros = data[~mask].copy()

In [11]:
from sklearn.model_selection import train_test_split
X_train_ones, X_test_ones, y_train_ones, y_test_ones = train_test_split(df_ones.drop(columns=['target']), df_ones['target'], test_size=0.3, random_state=42)

In [12]:
sampled_x_train_zeros, sampled_x_zeros, sampled_y_train_zeros, samples_y_zeros = train_test_split(df_zeros.drop(columns=['target']), df_zeros['target'], test_size=0.01, random_state=42)

In [13]:
X_TR_ZEROS, X_TEST_ZEROS, Y_TR_ZEROS, Y_TEST_ZEROS = train_test_split(sampled_x_zeros, samples_y_zeros, test_size=0.2, random_state=42)

In [14]:
# Features (X)
X_train = pd.concat([X_train_ones, X_TR_ZEROS], ignore_index=True)
X_test  = pd.concat([X_test_ones, X_TEST_ZEROS], ignore_index=True)

# Labels (y) - keep as DataFrame, not array
y_train = pd.concat([y_train_ones, Y_TR_ZEROS], ignore_index=True)
y_test  = pd.concat([y_test_ones, Y_TEST_ZEROS], ignore_index=True)

In [15]:
import torch

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()


In [16]:
X_test.shape

(920, 35)

In [17]:
images_test = []
images_train = []
for i in X_test['images_idx'].to_numpy():
    images_test.append(images[i])
for i in X_train['images_idx'].to_numpy():
    images_train.append(images[i])

In [18]:
from tensorflow import keras

2025-08-16 06:31:37.958256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755325898.143222     629 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755325898.195890     629 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

from torch.utils.data import DataLoader, Dataset

from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor()
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, pil_images, targets, X, transform=None):
        self.X = X
        self.images = pil_images
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        label = self.targets[idx]
        return image, self.X.iloc[idx].values, label

In [20]:
training = CustomDataset(images_train, y_train, X_train.drop(columns=['images_idx']), preprocess)
testing = CustomDataset(images_test, y_test, X_test.drop(columns=['images_idx']), preprocess)
train_dataloader = DataLoader(training, batch_size=128, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(testing, batch_size=128, shuffle=True, pin_memory=True)

In [21]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs)
        X.append(x)
        y.append(labels)
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_tab = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_tab = dataloader_to_numpy(test_dataloader)

In [22]:
import tensorflow as tf

In [24]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs)
        X.append(x)
        y.append(labels)
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_tab_ = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_tab_ = dataloader_to_numpy(test_dataloader)

In [31]:
seed_value = 42
tf.random.set_seed(seed_value)   
np.random.seed(seed_value)

In [28]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, pil_images, targets, X, transform=None):
        self.X = X
        self.images = pil_images
        self.targets = targets
        self.transform = transform
        assert len(self.images) == len(self.X) == len(self.targets), "Length mismatch"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        label = self.targets[idx]
        return image, self.X.iloc[idx].values, label

In [25]:
ones_index = list(X_train_ones['images_idx'])
# print(ones_index)

images_train_ones  = [images[i] for i in ones_index]
# print(images_train_ones)

print(len(images_train_ones))

275


In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array

# Initialize your ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

augmented_images_train_ones = []
augmented_ones_train= []
k = 0
for pil_img in images_train_ones:
    x = img_to_array(pil_img)
    x = x.reshape((1,) + x.shape)  # reshape for datagen
    i = 0
    for batch in datagen.flow(x, batch_size=1):
        augmented_images_train_ones.append(batch[0].astype(np.uint8))  # Store augmented image array
        i += 1
        augmented_ones_train.append(X_train_ones.iloc[k].values)
        if i >= 5:  # Number of augmentations per image
            break
    k+=1

In [27]:
augmented_ones_df = pd.DataFrame(augmented_ones_train, columns = X_train_ones.columns)

X_train_new = pd.concat([X_train, augmented_ones_df], ignore_index=True)

images_train_new = images_train.copy()

augmented_images_train_ones_pil = [
    Image.fromarray(arr).convert('RGB') for arr in augmented_images_train_ones
]

for img in augmented_images_train_ones_pil:
    images_train_new.append(img)


y_train_new = [i for i in y_train]

for j in range(0, len(augmented_images_train_ones)):
    y_train_new.append(1)

y_train_new = np.array([val for val in y_train_new]).flatten()



In [32]:
training = CustomDataset(images_train_new, y_train_new, X_train_new.drop(columns=['images_idx']), preprocess)
testing = CustomDataset(images_test, y_test, X_test.drop(columns=['images_idx']), preprocess)
train_dataloader = DataLoader(training, batch_size=128, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(testing, batch_size=128, shuffle=True, pin_memory=True)

In [33]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs)
        X.append(x)
        y.append(labels)
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_tab_ = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_tab_ = dataloader_to_numpy(test_dataloader)

In [34]:
X_train_img[0].shape

(224, 224, 3)

# VIT

In [30]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score

print("TF:", tf.__version__)
AUTOTUNE = tf.data.AUTOTUNE

# Image / model params
IMG_SIZE = (224, 224, 3)
PATCH_SIZE = 16                 # 224 / 16 => 14x14 = 196 tokens
NUM_PATCHES = (IMG_SIZE[0] // PATCH_SIZE) * (IMG_SIZE[1] // PATCH_SIZE)
EMBED_DIM = 256
NUM_HEADS = 8
MLP_DIM = 512
ENCODER_DEPTH = 8
DROP_RATE = 0.1

# Tabular params
TAB_DIM = 34                    # you said 34 columns

# Training
BATCH_SIZE = 32
EPOCHS = 20
LR = 3e-4
SEED = 42

# Mixed precision (optional, speed-up on GPU)
if tf.config.list_physical_devices("GPU"):
    tf.keras.mixed_precision.set_global_policy("mixed_float16")
    print("Using mixed precision")

TF: 2.18.0
Using mixed precision


In [50]:
def normalize_images(x):
    x = tf.convert_to_tensor(x)
    x = tf.image.convert_image_dtype(x, tf.float32)  # [0,1]
    return x

X_train_img_tf = normalize_images(X_train_img)
X_test_img_tf  = normalize_images(X_test_img)

# 2.2 Scale tabular with StandardScaler (fit on train only)
scaler = StandardScaler()
X_train_tab_scaled = scaler.fit_transform(X_train_tab_).astype("float32")
X_test_tab_scaled = scaler.transform(X_test_tab_).astype("float32")

# 2.3 Build tf.data datasets
def make_ds(imgs, tabs, ys, training: bool):
    ds_imgs = tf.data.Dataset.from_tensor_slices(imgs)
    ds_tabs = tf.data.Dataset.from_tensor_slices(tabs)
    ds_ys   = tf.data.Dataset.from_tensor_slices(ys.astype("float32"))
    ds = tf.data.Dataset.zip((ds_imgs, ds_tabs, ds_ys))

    def to_dict(img, tab, y):
        return {"image": img, "tab": tab}, y

    ds = ds.map(to_dict, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(X_train_img_tf, X_train_tab_scaled, y_train_, training=True)
val_ds   = make_ds(X_test_img_tf,  X_test_tab_scaled, y_test_,  training=False)

In [32]:
class PatchExtractor(layers.Layer):
    """Extract non-overlapping patches using a Conv2D."""
    def __init__(self, patch_size=PATCH_SIZE, embed_dim=EMBED_DIM, **kwargs):
        super().__init__(**kwargs)
        self.proj = layers.Conv2D(embed_dim, kernel_size=patch_size, strides=patch_size, padding="valid")

    def call(self, x):
        # x: [B, H, W, C] -> conv -> [B, H/P, W/P, EMBED]
        x = self.proj(x)
        # [B, h, w, EMBED] -> [B, h*w, EMBED]
        x = tf.reshape(x, [tf.shape(x)[0], -1, tf.shape(x)[-1]])
        return x  # tokens

class AddClassTokenAndPos(layers.Layer):
    def __init__(self, num_patches, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.cls = self.add_weight(
            name="cls",
            shape=(1, 1, embed_dim),
            initializer="zeros",
            trainable=True
        )
        self.pos = self.add_weight(
            name="pos",
            shape=(1, num_patches + 1, embed_dim),
            initializer="random_normal",
            trainable=True
        )

    def call(self, x):
        batch_size = tf.shape(x)[0]
        cls_tokens = tf.broadcast_to(self.cls, [batch_size, 1, tf.shape(x)[-1]])
        x = tf.concat([cls_tokens, x], axis=1)
        return x + self.pos

def transformer_encoder(x, num_heads=NUM_HEADS, mlp_dim=MLP_DIM, drop=DROP_RATE):
    # PreNorm -> MHA -> Drop -> Residual
    y = layers.LayerNormalization(epsilon=1e-6)(x)
    y = layers.MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(y, y)
    y = layers.Dropout(drop)(y)
    x = layers.Add()([x, y])

    # PreNorm -> MLP -> Drop -> Residual
    y = layers.LayerNormalization(epsilon=1e-6)(x)
    y = layers.Dense(mlp_dim, activation="gelu")(y)
    y = layers.Dropout(drop)(y)
    y = layers.Dense(x.shape[-1])(y)
    y = layers.Dropout(drop)(y)
    return layers.Add()([x, y])

In [47]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor()
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, pil_images, targets, X, transform=None):
        self.X = X
        self.images = pil_images
        self.targets = targets
        self.transform = transform
        assert len(self.images) == len(self.X) == len(self.targets), "Length mismatch"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        label = self.targets[idx]
        return image, self.X.iloc[idx].values, label

In [48]:
training = CustomDataset(images_train_new, y_train_new, X_train_new.drop(columns=['images_idx']), preprocess)
testing = CustomDataset(images_test, y_test, X_test.drop(columns=['images_idx']), preprocess)
train_dataloader = DataLoader(training, batch_size=128, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(testing, batch_size=128, shuffle=True, pin_memory=True)

In [49]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs.numpy())
        X.append(x.numpy())
        y.append(labels.numpy())
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_tab_ = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_tab_ = dataloader_to_numpy(test_dataloader)

In [33]:

def build_model():
    # Image branch (ViT)
    img_in = layers.Input(shape=IMG_SIZE, name="image")
    x = PatchExtractor(PATCH_SIZE, EMBED_DIM)(img_in)
    x = AddClassTokenAndPos(NUM_PATCHES, EMBED_DIM)(x)
    for _ in range(ENCODER_DEPTH):
        x = transformer_encoder(x, NUM_HEADS, MLP_DIM, DROP_RATE)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    cls_token = x[:, 0]                        # [B, D]
    img_feat = layers.Dropout(DROP_RATE, name="image_features")(cls_token)

    # Tabular branch (simple MLP)
    tab_in = layers.Input(shape=(TAB_DIM,), name="tab")
    t = layers.Dense(128, activation="relu")(tab_in)
    t = layers.BatchNormalization()(t)
    t = layers.Dropout(0.2)(t)
    t = layers.Dense(128, activation="relu")(t)
    t = layers.BatchNormalization()(t)
    t = layers.Dropout(0.2)(t)

    # Fuse
    fused = layers.Concatenate(name="fusion")([img_feat, t])
    z = layers.Dense(256, activation="relu")(fused)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.3)(z)

    # Binary output (ensure float32 to avoid mixed precision issues on loss/metrics)
    out = layers.Dense(1, activation="sigmoid", dtype="float32", name="pred")(z)

    return keras.Model(inputs={"image": img_in, "tab": tab_in}, outputs=out, name="ViT_Tab_Fusion")

model = build_model()
model.summary()


Model: "ViT_Tab_Fusion"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 224, 224,  │          0 │ image[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_extractor     │ (None, None, 256) │    196,864 │ cast[0][0]        │
│ (PatchExtractor)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_class_token_an… │ (None, 197, 256)  │     50,688 │ patch_extractor[… │
│ (AddClassTokenAndP… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 197, 256)  │        512 │ add_class_token_… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 197, 256)  │  2,103,552 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 197, 256)  │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 197, 256)  │          0 │ add_class_token_… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 197, 256)  │        512 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 197, 512)  │    131,584 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 197, 512)  │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 197, 256)  │    131,328 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 197, 256)  │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 197, 256)  │          0 │ add[0][0],        │
│                     │                   │            │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 197, 256)  │        512 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 197, 256)  │  2,103,552 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 197, 256)  │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 197, 256)  │          0 │ add_1[0][0],      │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 197, 256)  │        512 │ add_2[0][0]     

 Total params: 19,309,825 (73.66 MB)

 Trainable params: 19,308,801 (73.66 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [34]:
class F1AtThreshold(keras.metrics.Metric):
    """Compute F1 with fixed threshold (default 0.5)."""
    def __init__(self, threshold=0.5, name="f1", **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
        y_pred = tf.cast(tf.reshape(y_pred, [-1]) >= self.threshold, tf.float32)

        tp = tf.reduce_sum(y_pred * y_true)
        fp = tf.reduce_sum(y_pred * (1.0 - y_true))
        fn = tf.reduce_sum((1.0 - y_pred) * y_true)

        self.tp.assign_add(tp)
        self.fp.assign_add(fp)
        self.fn.assign_add(fn)

    def result(self):
        precision = self.tp / (self.tp + self.fp + 1e-7)
        recall = self.tp / (self.tp + self.fn + 1e-7)
        f1 = 2.0 * precision * recall / (precision + recall + 1e-7)
        return f1

    def reset_states(self):
        for v in self.variables:
            v.assign(0.0)

metrics = [
    keras.metrics.Precision(name="precision", thresholds=0.5),
    keras.metrics.Recall(name="recall",     thresholds=0.5),
    keras.metrics.AUC(name="auc_roc", curve="ROC"),
    F1AtThreshold(threshold=0.5, name="f1"),
]

model.compile(
    optimizer=keras.optimizers.Adam(LR),
    loss="binary_crossentropy",
    metrics=metrics,
)

In [55]:
ckpt_path = "vit_image.keras"
cbs = [
    callbacks.ModelCheckpoint(ckpt_path, monitor="val_auc_roc", mode="max",
                              save_best_only=True, verbose=1),
    callbacks.EarlyStopping(monitor="val_auc_roc", mode="max",
                            patience=5, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
]

history = model.fit(
    train_ds,
    validation_data=val_ds,   # per your request, using test as validation
    epochs=EPOCHS,
    callbacks=cbs,
    verbose=1
)

Epoch 1/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - auc_roc: 0.7699 - f1: 0.6228 - loss: 0.6182 - precision: 0.5660 - recall: 0.6956
Epoch 1: val_auc_roc improved from -inf to 0.87410, saving model to vit_image.keras
152/152 ━━━━━━━━━━━━━━━━━━━━ 128s 432ms/step - auc_roc: 0.7703 - f1: 0.6232 - loss: 0.6176 - precision: 0.5665 - recall: 0.6958 - val_auc_roc: 0.8741 - val_f1: 0.5027 - val_loss: 0.2657 - val_precision: 0.7077 - val_recall: 0.3898 - learning_rate: 3.0000e-04
Epoch 2/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - auc_roc: 0.8798 - f1: 0.7225 - loss: 0.4364 - precision: 0.7032 - recall: 0.7446
Epoch 2: val_auc_roc improved from 0.87410 to 0.88343, saving model to vit_image.keras
152/152 ━━━━━━━━━━━━━━━━━━━━ 31s 201ms/step - auc_roc: 0.8798 - f1: 0.7226 - loss: 0.4363 - precision: 0.7034 - recall: 0.7446 - val_auc_roc: 0.8834 - val_f1: 0.5859 - val_loss: 0.3878 - val_precision: 0.4860 - val_recall: 0.7373 - learning_rate: 3.0000e-04
Epoch 3/20
152/152 ━━━━━━━━━━━━━━━━━━━

In [39]:
final = tf.keras.models.load_model(
    "/kaggle/input/intras-1/best_vit_tab.keras",
    custom_objects={
        "PatchExtractor": PatchExtractor,
        "AddClassTokenAndPos": AddClassTokenAndPos,
        "transformer_encoder": transformer_encoder
    },
    compile=False  # Skip loading loss/metrics state
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'patch_extractor_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [44]:
X_train_img_tf.shape

TensorShape([4855, 224, 224, 3])

In [45]:
from sklearn.metrics import classification_report
y_pred = final.predict([X_test_img_tf,  X_test_tab_scaled])
for threshold in np.linspace(0, 0.5, 20):
    y_ = y_pred > threshold
    print(f'threshold = {threshold}')
    print(classification_report(y_test_, y_))

NameError: name 'final' is not defined

In [35]:
class F1ScoreWithThreshold(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.3, name="f1_score_with_threshold", **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * p * r / (p + r + 1e-8)  # F1 formula with epsilon to prevent divide by zero

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [36]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor()
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, pil_images, targets, X, transform=None):
        self.X = X
        self.images = pil_images
        self.targets = targets
        self.transform = transform
        assert len(self.images) == len(self.X) == len(self.targets), "Length mismatch"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        label = self.targets[idx]
        return image, self.X.iloc[idx].values, label

In [60]:
training = CustomDataset(images_train_new, y_train_new, X_train_new.drop(columns=['images_idx']), preprocess)
testing = CustomDataset(images_test, y_test, X_test.drop(columns=['images_idx']), preprocess)
train_dataloader = DataLoader(training, batch_size=128, shuffle = True, pin_memory = True)
test_dataloader = DataLoader(testing, batch_size=128, shuffle = True, pin_memory = True)

In [61]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs)
        X.append(x)
        y.append(labels)
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_tab_ = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_tab_ = dataloader_to_numpy(test_dataloader)

In [37]:
class F1ScoreWithThreshold(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.3, name="f1_score_with_threshold", **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * p * r / (p + r + 1e-8)  # F1 formula with epsilon to prevent divide by zero

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [39]:
from tensorflow.keras.applications import ResNet50, EfficientNetV2M
from tensorflow.keras.layers import (
    Input, Dense, Concatenate, BatchNormalization, Dropout, GlobalAveragePooling2D, MaxPooling2D, Flatten
)
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import AdamW, Adam
from tensorflow.keras.initializers import HeNormal
import tensorflow as tf

# Custom Focal Loss and Custom Metric
def binary_focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * tf.pow(y_pred, gamma) * (1 - y_true)
        return tf.reduce_mean(weight * cross_entropy)
    return loss

class RecallWithThreshold(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.3, name="recall_with_threshold", **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.recall.result()

    def reset_states(self):
        self.recall.reset_states()

# -------- Inputs -------- 
image_input = Input(shape=(128, 128, 3), name='image_input')
num_input = Input(shape=(34,), name='num_input')

# -------- Image branch with ResNet50 --------
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)
base_model.trainable = False  # freeze ResNet weights

x1 = base_model.output
x1 = MaxPooling2D((2, 2))(x1)
x1 = Flatten()(x1)

x1 = Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)

x1 = Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)

x1 = Dense(16, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4))(x1)

# -------- Numerical branch --------
x2 = BatchNormalization()(num_input)
x2 = Dense(48, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.2)(x2)

x2 = Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.2)(x2)

x2 = Dense(16, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4))(x2)

# -------- Merge branches --------
merged = Concatenate()([x1, x2])

merged = BatchNormalization()(merged)
merged = Dropout(0.2)(merged)
merged = Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4))(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.2)(merged)
merged = Dense(16, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-4))(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.2)(merged)
output = Dense(1, activation='sigmoid')(merged)

# -------- Build and Compile Model --------
model = Model(inputs=[image_input, num_input], outputs=output)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss=binary_focal_loss(),
    metrics=['precision', F1ScoreWithThreshold(), RecallWithThreshold()]
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_5 (Cast)       │ (None, 128, 128,  │          0 │ image_input[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ cast_5[0][0]      │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c

 Total params: 24,122,905 (92.02 MB)

 Trainable params: 534,549 (2.04 MB)

 Non-trainable params: 23,588,356 (89.98 MB)

# LOADING MODELS 

In [41]:
vit = tf.keras.models.load_model(
    "/kaggle/input/intras-1/best_vit_tab.keras",
    custom_objects={
        "PatchExtractor": PatchExtractor,
        "AddClassTokenAndPos": AddClassTokenAndPos,
        "transformer_encoder": transformer_encoder
    },
    compile=False  # Skip loading loss/metrics state
)
resnet = tf.keras.models.load_model(
    "/kaggle/input/intras-1/RESNET_MODEL.h5",
    custom_objects={
        "binary_focal_loss" : binary_focal_loss()
    },
    compile=False  # Skip loading loss/metrics state
)
effnet = tf.keras.models.load_model(
    "/kaggle/input/intras-1/EFFICIENT_NET.h5",
    custom_objects={
        "binary_focal_loss" : binary_focal_loss()
    },
    compile=False  # Skip loading loss/metrics state
)

In [46]:
vit_im_ex = tf.keras.models.load_model(
    "/kaggle/working/vit_image.keras",
    custom_objects={
        "PatchExtractor": PatchExtractor,
        "AddClassTokenAndPos": AddClassTokenAndPos,
        "transformer_encoder": transformer_encoder
    },
    compile=False  # Skip loading loss/metrics state
)
vit_im_ol = tf.keras.models.Model(
    inputs=vit_im_ex.input,
    outputs=vit_im_ex.get_layer("image_features").output
)
X_train_vit_im = vit_im_ol.predict([X_train_img_tf, X_train_tab_scaled])
X_test_vit_im = vit_im_ol.predict([X_test_img_tf, X_test_tab_scaled])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'patch_extractor_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: {'image': 'image', 'tab': 'tab'}
Received: inputs=('Tensor(shape=(32, 224, 224, 3))', 'Tensor(shape=(32, 34))')
  warnings.warn(msg)
I0000 00:00:1755326203.538451     666 service.cc:148] XLA service 0x789da8008cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755326203.539040     666 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capabi

  3/152 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step 

I0000 00:00:1755326206.824357     666 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


151/152 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: {'image': 'image', 'tab': 'tab'}
Received: inputs=('Tensor(shape=(None, 224, 224, 3))', 'Tensor(shape=(None, 34))')
  warnings.warn(msg)


152/152 ━━━━━━━━━━━━━━━━━━━━ 15s 66ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 111ms/step


In [57]:
# vit output
vit_ol = tf.keras.models.Model(
    inputs=vit.input,
    outputs=vit.get_layer("batch_normalization_21").output
)
X_train_vit = vit_ol.predict([X_train_img_tf, X_train_tab_scaled])
X_test_vit = vit_ol.predict([X_test_img_tf, X_test_tab_scaled])

152/152 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step


In [59]:
# resnet output 
resnet_ol = tf.keras.models.Model(
    inputs=resnet.input,
    outputs=resnet.get_layer("dense_11").output
)
X_train_resnet = resnet_ol.predict([X_train_img, X_train_tab_])
X_test_resnet = resnet_ol.predict([X_test_img, X_test_tab_])

152/152 ━━━━━━━━━━━━━━━━━━━━ 14s 54ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step


In [62]:
# effnet output
effnet_ol = tf.keras.models.Model(
    inputs=effnet.input,
    outputs=effnet.get_layer("dense_20").output
)
X_train_effnet = effnet_ol.predict([X_train_img, X_train_tab_])
X_test_effnet = effnet_ol.predict([X_test_img, X_test_tab_])

152/152 ━━━━━━━━━━━━━━━━━━━━ 35s 146ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 231ms/step


In [63]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking

In [65]:
!pip install autogluon

  Using cached autogluon-1.4.0-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.core-1.4.0-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.features-1.4.0-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.tabular-1.4.0-py3-none-any.whl.metadata (16 kB)
  Using cached autogluon.multimodal-1.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached autogluon.timeseries-1.4.0-py3-none-any.whl.metadata (12 kB)
  Using cached scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached autogluon.common-1.4.0-py3-none-any.whl.metadata (11 kB)
  Using cached ray-2.44.1-cp311-cp311-manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached lightning-2.5.3-py3-none-any.whl.metadata (39 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached seqeval-1.2.2.tar.gz (

In [66]:
import sys
print(sys.version)


3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [67]:
!pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 8.9 MB/s eta 0:00:00:00:01


In [70]:
print(X_train_new.shape)
print(len(y_train_new))

(4855, 35)
4855


In [71]:
from flaml import AutoML

automl = AutoML()

automl_settings = {
    "time_budget": 60,
    "task": "classification",
    "metric": "accuracy",
    "log_file_name": "flaml.log"
}

In [72]:
automl.fit(X_train=X_train_new, y_train = y_train_, **automl_settings)

[flaml.automl.logger: 08-16 07:13:35] {1752} INFO - task = classification
[flaml.automl.logger: 08-16 07:13:35] {1763} INFO - Evaluation method: holdout
[flaml.automl.logger: 08-16 07:13:35] {1862} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 08-16 07:13:35] {1979} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'catboost', 'lrl1']
[flaml.automl.logger: 08-16 07:13:35] {2282} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 08-16 07:13:35] {2417} INFO - Estimated sufficient time budget=544s. Estimated necessary time budget=13s.
[flaml.automl.logger: 08-16 07:13:35] {2466} INFO -  at 0.3s,	estimator lgbm's best error=0.3402,	best estimator lgbm's best error=0.3402
[flaml.automl.logger: 08-16 07:13:35] {2282} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 08-16 07:13:35] {2466} INFO -  at 0.4s,	estimator lgbm's best error=0.3402,	best estimator lgbm's best error=0.3402
[flaml.auto

In [74]:
# Predict class labels
y_pred = automl.predict(X_test)

# Predict probabilities (for classification)
y_pred_proba = automl.predict_proba(X_test)


In [75]:
print(automl.model)


In [79]:
from sklearn.metrics import classification_report

# Step 1: Get predicted probabilities (already done)
# y_pred_proba = automl.predict_proba(X_test)

# Step 2: Apply threshold to get binary labels
threshold = 0.3
y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)  # For binary classification

# Step 3: Print the classification metrics
print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

         0.0     0.8828    0.1409    0.2430       802
         1.0     0.1301    0.8729    0.2264       118

    accuracy                         0.2348       920
   macro avg     0.5064    0.5069    0.2347       920
weighted avg     0.7863    0.2348    0.2409       920



### 

In [81]:
len(X_train_tab_scaled)


4855

In [89]:
# ==============================
# Weighted-block Stacking w/ Optuna
# Base learners: XGB, CatBoost, Logistic Regression
# Meta learner: Logistic Regression
# ==============================

import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score  # change to your metric if needed
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# --------------------------------------------------
# Required inputs 
# Shapes: (n_train, d_*), (n_test, d_*)
# --------------------------------------------------
# X_train_effnet, X_test_effnet
# X_train_resnet, X_test_resnet
# X_train_vit_im,   X_test_vit_im
# X_train_tab_scaled,    X_test_tab_scaled
# y_train_,     y_test_

# ====== Helpers ======

def weighted_concat(blocks, weights):
    """Multiply each feature block by its scalar weight and concatenate horizontally."""
    assert len(blocks) == len(weights)
    w_blocks = [w * b for w, b in zip(weights, blocks)]
    return np.hstack(w_blocks)

def get_oof_preds(model, X, y, X_test, n_splits=5, random_state=42):
    """Return OOF predictions for stacking + mean test predictions."""
    oof = np.zeros(len(X))
    test_fold = np.zeros((len(X_test), n_splits))

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_va, y_va = X[va_idx], y[va_idx]

        model.fit(X_tr, y_tr)
        oof[va_idx] = model.predict_proba(X_va)[:, 1]
        test_fold[:, fold] = model.predict_proba(X_test)[:, 1]

    return oof.reshape(-1, 1), test_fold.mean(axis=1).reshape(-1, 1)

# ====== Optuna Objective ======

def objective(trial):
    # -----------------------------
    # 1) Suggest weights in [1, 5]
    # -----------------------------
    # XGB input weights
    a1 = trial.suggest_float("xgb_w_effnet", 1.0, 5.0)
    a2 = trial.suggest_float("xgb_w_resnet", 1.0, 5.0)
    a3 = trial.suggest_float("xgb_w_vit",    1.0, 5.0)
    a4 = trial.suggest_float("xgb_w_tab",    1.0, 5.0)

    # CatBoost input weights
    b1 = trial.suggest_float("cat_w_effnet", 1.0, 5.0)
    b2 = trial.suggest_float("cat_w_resnet", 1.0, 5.0)
    b3 = trial.suggest_float("cat_w_vit",    1.0, 5.0)
    b4 = trial.suggest_float("cat_w_tab",    1.0, 5.0)

    # LR input weights
    c1 = trial.suggest_float("lr_w_effnet",  1.0, 5.0)
    c2 = trial.suggest_float("lr_w_resnet",  1.0, 5.0)
    c3 = trial.suggest_float("lr_w_vit",     1.0, 5.0)
    c4 = trial.suggest_float("lr_w_tab",     1.0, 5.0)

    # -----------------------------
    # 2) Suggest model hyperparams
    # -----------------------------
    # XGB
    xgb_params = {
        "n_estimators": 300,
        "max_depth": 4,
        "learning_rate": 0.05,
        "subsample": 0.85,
        "colsample_bytree": 0.7,
        "reg_lambda": 2e-3,
        "random_state": 42,
        "eval_metric": "logloss",
        "use_label_encoder": False,
        "n_jobs": -1,
    }

    # CatBoost (use silent training)
    cat_params = {
        "iterations": 500,
        "depth": 5,
        "learning_rate": 2e-3,
        "l2_leaf_reg": 3,
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "random_seed": 42,
        "verbose": False
    }

    # LR (base learners & meta learner use possibly different Cs)
    lr_C_base = trial.suggest_float("lr_base_C", 1e-3, 10.0, log=True)
    lr_solver_base = trial.suggest_categorical("lr_base_solver", ["lbfgs", "liblinear", "saga"])

    lr_C_meta = trial.suggest_float("lr_meta_C", 1e-3, 10.0, log=True)
    lr_solver_meta = trial.suggest_categorical("lr_meta_solver", ["lbfgs", "liblinear", "saga"])

    # -----------------------------
    # 3) Build weighted inputs
    # -----------------------------
    X_xgb_train = weighted_concat(
        [X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
        [a1, a2, a3, a4]
    )
    X_xgb_test = weighted_concat(
        [X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
        [a1, a2, a3, a4]
    )

    X_cat_train = weighted_concat(
        [X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
        [b1, b2, b3, b4]
    )
    X_cat_test = weighted_concat(
        [X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
        [b1, b2, b3, b4]
    )

    X_lr_train = weighted_concat(
        [X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
        [c1, c2, c3, c4]
    )
    X_lr_test = weighted_concat(
        [X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
        [c1, c2, c3, c4]
    )

    # -----------------------------
    # 4) Define base learners
    # -----------------------------
    base_xgb = XGBClassifier(**xgb_params)
    base_cat = CatBoostClassifier(**cat_params)
    base_lr  = LogisticRegression(
        C=lr_C_base, solver=lr_solver_base, max_iter=1000, n_jobs=None if lr_solver_base=="liblinear" else -1
    )

    # -----------------------------
    # 5) OOF predictions for stacking
    # -----------------------------
    oof_xgb, test_xgb = get_oof_preds(base_xgb, X_xgb_train, y_train_, X_xgb_test)
    oof_cat, test_cat = get_oof_preds(base_cat, X_cat_train, y_train_, X_cat_test)
    oof_lr,  test_lr  = get_oof_preds(base_lr,  X_lr_train,  y_train_, X_lr_test)

    # Stack into meta features
    meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
    meta_test  = np.hstack([test_xgb, test_cat, test_lr])

    # -----------------------------
    # 6) Meta learner (LR)
    # -----------------------------
    meta_model = LogisticRegression(
        C=lr_C_meta, solver=lr_solver_meta, max_iter=1000,
        n_jobs=None if lr_solver_meta=="liblinear" else -1
    )
    meta_model.fit(meta_train, y_train_)
    meta_probs = meta_model.predict_proba(meta_test)[:, 1]

    # -----------------------------
    # 7) Metric (AUC by default)
    #     - If you want recall of minority class instead:
    #         from sklearn.metrics import recall_score
    #         preds = (meta_probs >= 0.5).astype(int)
    #         return recall_score(y_test_, preds, pos_label=1)
    # -----------------------------
    preds = (meta_probs >= 0.3).astype(int)  # Threshold at 0.3
    recall = recall_score(y_test_, preds, pos_label=1)
    return recall

# ====== Run Optuna ======
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)   # increase n_trials for better search

print("Best AUC:", study.best_value)
print("Best params:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")

# ====== Train final model with best params ======
best = study.best_params

# Rebuild weights
a1, a2, a3, a4 = best["xgb_w_effnet"], best["xgb_w_resnet"], best["xgb_w_vit"], best["xgb_w_tab"]
b1, b2, b3, b4 = best["cat_w_effnet"], best["cat_w_resnet"], best["cat_w_vit"], best["cat_w_tab"]
c1, c2, c3, c4 = best["lr_w_effnet"],  best["lr_w_resnet"],  best["lr_w_vit"],  best["lr_w_tab"]

# Rebuild features
X_xgb_train = weighted_concat([effnet_train, resnet_train, vit_train, tab_train], [a1, a2, a3, a4])
X_xgb_test  = weighted_concat([effnet_test,  resnet_test,  vit_test,  tab_test],  [a1, a2, a3, a4])

X_cat_train = weighted_concat([effnet_train, resnet_train, vit_train, tab_train], [b1, b2, b3, b4])
X_cat_test  = weighted_concat([effnet_test,  resnet_test,  vit_test,  tab_test],  [b1, b2, b3, b4])

X_lr_train  = weighted_concat([effnet_train, resnet_train, vit_train, tab_train], [c1, c2, c3, c4])
X_lr_test   = weighted_concat([effnet_test,  resnet_test,  vit_test,  tab_test],  [c1, c2, c3, c4])

# Final models
final_xgb = XGBClassifier(
    n_estimators=best["xgb_n_estimators"],
    max_depth=best["xgb_max_depth"],
    learning_rate=best["xgb_lr"],
    subsample=best["xgb_subsample"],
    colsample_bytree=best["xgb_colsample"],
    reg_lambda=best["xgb_reg_lambda"],
    random_state=42, eval_metric="logloss", use_label_encoder=False, n_jobs=-1
)
final_cat = CatBoostClassifier(
    iterations=best["cat_iterations"], depth=best["cat_depth"],
    learning_rate=best["cat_lr"], l2_leaf_reg=best["cat_l2_leaf_reg"],
    loss_function="Logloss", eval_metric="AUC", random_seed=42, verbose=False
)
final_lr_base = LogisticRegression(
    C=best["lr_base_C"], solver=best["lr_base_solver"], max_iter=1000,
    n_jobs=None if best["lr_base_solver"]=="liblinear" else -1
)

# OOF again to build full meta features (train on all folds)
oof_xgb, test_xgb = get_oof_preds(final_xgb, X_xgb_train, y_train_, X_xgb_test)
oof_cat, test_cat = get_oof_preds(final_cat, X_cat_train, y_train_, X_cat_test)
oof_lr,  test_lr  = get_oof_preds(final_lr_base, X_lr_train, y_train_, X_lr_test)

meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
meta_test  = np.hstack([test_xgb, test_cat, test_lr])

final_meta = LogisticRegression(
    C=best["lr_meta_C"], solver=best["lr_meta_solver"], max_iter=1000,
    n_jobs=None if best["lr_meta_solver"]=="liblinear" else -1
)
final_meta.fit(meta_train, y_train_)
final_probs = final_meta.predict_proba(meta_test)[:, 1]

# Report
from sklearn.metrics import roc_auc_score, classification_report
final_auc = roc_auc_score(y_test_, final_probs)
final_preds = (final_probs >= 0.3).astype(int)
print(f"\nFinal AUC: {final_auc:.4f}")
print(classification_report(y_test_, final_preds, digits=4))


[I 2025-08-16 07:43:52,564] A new study created in memory with name: no-name-b52bba5d-3cf3-4c56-831e-6647f74f3189
[W 2025-08-16 07:44:38,921] Trial 0 failed with parameters: {'xgb_w_effnet': 4.660558757654176, 'xgb_w_resnet': 1.026490307067438, 'xgb_w_vit': 4.621715800737713, 'xgb_w_tab': 1.2590703130942704, 'cat_w_effnet': 4.049580052978702, 'cat_w_resnet': 4.598132819712559, 'cat_w_vit': 1.2712661756691355, 'cat_w_tab': 3.0810582848374977, 'lr_w_effnet': 3.254347329143679, 'lr_w_resnet': 1.247330012159305, 'lr_w_vit': 1.3038227691581863, 'lr_w_tab': 4.968181531825035, 'lr_base_C': 1.1874525731286782, 'lr_base_solver': 'liblinear', 'lr_meta_C': 0.14354474451353696, 'lr_meta_solver': 'saga'} because of the following error: NameError("name 'recall_score' is not defined").
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ip

NameError: name 'recall_score' is not defined

In [83]:
best = study.best_params  # Already done after running Optuna


In [84]:
X_xgb_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], 
                              [best["xgb_w_effnet"], best["xgb_w_resnet"], best["xgb_w_vit"], best["xgb_w_tab"]])
X_xgb_test  = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], 
                              [best["xgb_w_effnet"], best["xgb_w_resnet"], best["xgb_w_vit"], best["xgb_w_tab"]])

X_cat_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], 
                              [best["cat_w_effnet"], best["cat_w_resnet"], best["cat_w_vit"], best["cat_w_tab"]])
X_cat_test  = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], 
                              [best["cat_w_effnet"], best["cat_w_resnet"], best["cat_w_vit"], best["cat_w_tab"]])

X_lr_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], 
                             [best["lr_w_effnet"], best["lr_w_resnet"], best["lr_w_vit"], best["lr_w_tab"]])
X_lr_test  = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], 
                             [best["lr_w_effnet"], best["lr_w_resnet"], best["lr_w_vit"], best["lr_w_tab"]])


In [85]:
final_xgb = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.7,
    reg_lambda=2e-3,
    random_state=42,
    eval_metric="logloss",
    use_label_encoder=False,
    n_jobs=-1
)
final_cat = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=2e-3,
    l2_leaf_reg=3,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=False
)
final_lr_base = LogisticRegression(
    C=best["lr_base_C"], 
    solver=best["lr_base_solver"], 
    max_iter=1000, 
    n_jobs=None if best["lr_base_solver"]=="liblinear" else -1
)


In [86]:
oof_xgb, test_xgb = get_oof_preds(final_xgb, X_xgb_train, y_train_, X_xgb_test)
oof_cat, test_cat = get_oof_preds(final_cat, X_cat_train, y_train_, X_cat_test)
oof_lr,  test_lr  = get_oof_preds(final_lr_base, X_lr_train, y_train_, X_lr_test)

meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
meta_test  = np.hstack([test_xgb, test_cat, test_lr])


In [87]:
final_meta = LogisticRegression(
    C=best["lr_meta_C"], solver=best["lr_meta_solver"], max_iter=1000,
    n_jobs=None if best["lr_meta_solver"]=="liblinear" else -1
)
final_meta.fit(meta_train, y_train_)
final_probs = final_meta.predict_proba(meta_test)[:, 1]


In [88]:
from sklearn.metrics import roc_auc_score, classification_report
final_auc = roc_auc_score(y_test_, final_probs)
final_preds = (final_probs >= 0.3).astype(int)
print(f"\nFinal AUC: {final_auc:.4f}")
print(classification_report(y_test_, final_preds, digits=4))



Final AUC: 0.9204
              precision    recall  f1-score   support

         0.0     0.9496    0.9626    0.9560       802
         1.0     0.7196    0.6525    0.6844       118

    accuracy                         0.9228       920
   macro avg     0.8346    0.8076    0.8202       920
weighted avg     0.9201    0.9228    0.9212       920



In [90]:
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, roc_auc_score, classification_report
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# --------------------------------------------------
# Required inputs (replace with your data arrays)
# Shapes: (n_train, d_*), (n_test, d_*)
# --------------------------------------------------
# X_train_effnet, X_test_effnet
# X_train_resnet, X_test_resnet
# X_train_vit_im,   X_test_vit_im
# X_train_tab_scaled,    X_test_tab_scaled
# y_train_,     y_test_
# --------------------------------------------------

def weighted_concat(blocks, weights):
    """Multiply each feature block by its scalar weight and concatenate horizontally."""
    assert len(blocks) == len(weights)
    w_blocks = [w * b for w, b in zip(weights, blocks)]
    return np.hstack(w_blocks)

def get_oof_preds(model, X, y, X_test, n_splits=5, random_state=42):
    """Return OOF predictions for stacking + mean test predictions."""
    oof = np.zeros(len(X))
    test_fold = np.zeros((len(X_test), n_splits))
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_va, y_va = X[va_idx], y[va_idx]
        model.fit(X_tr, y_tr)
        oof[va_idx] = model.predict_proba(X_va)[:, 1]
        test_fold[:, fold] = model.predict_proba(X_test)[:, 1]
    return oof.reshape(-1, 1), test_fold.mean(axis=1).reshape(-1, 1)

def objective(trial):
    # Suggest weights for each feature block for each model
    a1 = trial.suggest_float("xgb_w_effnet", 1.0, 5.0)
    a2 = trial.suggest_float("xgb_w_resnet", 1.0, 5.0)
    a3 = trial.suggest_float("xgb_w_vit", 1.0, 5.0)
    a4 = trial.suggest_float("xgb_w_tab", 1.0, 5.0)

    b1 = trial.suggest_float("cat_w_effnet", 1.0, 5.0)
    b2 = trial.suggest_float("cat_w_resnet", 1.0, 5.0)
    b3 = trial.suggest_float("cat_w_vit", 1.0, 5.0)
    b4 = trial.suggest_float("cat_w_tab", 1.0, 5.0)

    c1 = trial.suggest_float("lr_w_effnet", 1.0, 5.0)
    c2 = trial.suggest_float("lr_w_resnet", 1.0, 5.0)
    c3 = trial.suggest_float("lr_w_vit", 1.0, 5.0)
    c4 = trial.suggest_float("lr_w_tab", 1.0, 5.0)

    # Suggest hyperparameters for logistic regression base and meta learners
    lr_C_base = trial.suggest_float("lr_base_C", 1e-3, 10.0, log=True)
    lr_solver_base = trial.suggest_categorical("lr_base_solver", ["lbfgs", "liblinear", "saga"])
    lr_C_meta = trial.suggest_float("lr_meta_C", 1e-3, 10.0, log=True)
    lr_solver_meta = trial.suggest_categorical("lr_meta_solver", ["lbfgs", "liblinear", "saga"])

    # XGB fixed params
    xgb_params = {
        "n_estimators": 300,
        "max_depth": 4,
        "learning_rate": 0.05,
        "subsample": 0.85,
        "colsample_bytree": 0.7,
        "reg_lambda": 2e-3,
        "random_state": 42,
        "eval_metric": "logloss",
        "use_label_encoder": False,
        "n_jobs": -1,
    }

    # CatBoost fixed params
    cat_params = {
        "iterations": 500,
        "depth": 5,
        "learning_rate": 2e-3,
        "l2_leaf_reg": 3,
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "random_seed": 42,
        "verbose": False
    }

    # Build weighted inputs for base models
    X_xgb_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [a1, a2, a3, a4])
    X_xgb_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [a1, a2, a3, a4])

    X_cat_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [b1, b2, b3, b4])
    X_cat_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [b1, b2, b3, b4])

    X_lr_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [c1, c2, c3, c4])
    X_lr_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [c1, c2, c3, c4])

    # Initialize base learners
    base_xgb = XGBClassifier(**xgb_params)
    base_cat = CatBoostClassifier(**cat_params)
    base_lr = LogisticRegression(
        C=lr_C_base, solver=lr_solver_base,
        max_iter=1000,
        n_jobs=None if lr_solver_base == "liblinear" else -1
    )

    # Get out-of-fold predictions and test predictions
    oof_xgb, test_xgb = get_oof_preds(base_xgb, X_xgb_train, y_train_, X_xgb_test)
    oof_cat, test_cat = get_oof_preds(base_cat, X_cat_train, y


SyntaxError: incomplete input (1610422388.py, line 109)

In [91]:
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, roc_auc_score, classification_report
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# --------------------------------------------------
# Required inputs (replace with your data arrays)
# Shapes: (n_train, d_*), (n_test, d_*)
# --------------------------------------------------
# X_train_effnet, X_test_effnet
# X_train_resnet, X_test_resnet
# X_train_vit_im,   X_test_vit_im
# X_train_tab_scaled,    X_test_tab_scaled
# y_train_,     y_test_
# --------------------------------------------------

def weighted_concat(blocks, weights):
    """Multiply each feature block by its scalar weight and concatenate horizontally."""
    assert len(blocks) == len(weights)
    w_blocks = [w * b for w, b in zip(weights, blocks)]
    return np.hstack(w_blocks)

def get_oof_preds(model, X, y, X_test, n_splits=5, random_state=42):
    """Return OOF predictions for stacking + mean test predictions."""
    oof = np.zeros(len(X))
    test_fold = np.zeros((len(X_test), n_splits))
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
        X_tr, y_tr = X[tr_idx], y[tr_idx]
        X_va, y_va = X[va_idx], y[va_idx]
        model.fit(X_tr, y_tr)
        oof[va_idx] = model.predict_proba(X_va)[:, 1]
        test_fold[:, fold] = model.predict_proba(X_test)[:, 1]
    return oof.reshape(-1, 1), test_fold.mean(axis=1).reshape(-1, 1)

def objective(trial):
    # Suggest weights for each feature block for each model
    a1 = trial.suggest_float("xgb_w_effnet", 1.0, 5.0)
    a2 = trial.suggest_float("xgb_w_resnet", 1.0, 5.0)
    a3 = trial.suggest_float("xgb_w_vit", 1.0, 5.0)
    a4 = trial.suggest_float("xgb_w_tab", 1.0, 5.0)

    b1 = trial.suggest_float("cat_w_effnet", 1.0, 5.0)
    b2 = trial.suggest_float("cat_w_resnet", 1.0, 5.0)
    b3 = trial.suggest_float("cat_w_vit", 1.0, 5.0)
    b4 = trial.suggest_float("cat_w_tab", 1.0, 5.0)

    c1 = trial.suggest_float("lr_w_effnet", 1.0, 5.0)
    c2 = trial.suggest_float("lr_w_resnet", 1.0, 5.0)
    c3 = trial.suggest_float("lr_w_vit", 1.0, 5.0)
    c4 = trial.suggest_float("lr_w_tab", 1.0, 5.0)

    # Suggest hyperparameters for logistic regression base and meta learners
    lr_C_base = trial.suggest_float("lr_base_C", 1e-3, 10.0, log=True)
    lr_solver_base = trial.suggest_categorical("lr_base_solver", ["lbfgs", "liblinear", "saga"])
    lr_C_meta = trial.suggest_float("lr_meta_C", 1e-3, 10.0, log=True)
    lr_solver_meta = trial.suggest_categorical("lr_meta_solver", ["lbfgs", "liblinear", "saga"])

    # XGB fixed params
    xgb_params = {
        "n_estimators": 300,
        "max_depth": 4,
        "learning_rate": 0.05,
        "subsample": 0.85,
        "colsample_bytree": 0.7,
        "reg_lambda": 2e-3,
        "random_state": 42,
        "eval_metric": "logloss",
        "use_label_encoder": False,
        "n_jobs": -1,
    }

    # CatBoost fixed params
    cat_params = {
        "iterations": 500,
        "depth": 5,
        "learning_rate": 2e-3,
        "l2_leaf_reg": 3,
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "random_seed": 42,
        "verbose": False
    }

    # Build weighted inputs for base models
    X_xgb_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [a1, a2, a3, a4])
    X_xgb_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [a1, a2, a3, a4])

    X_cat_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [b1, b2, b3, b4])
    X_cat_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [b1, b2, b3, b4])

    X_lr_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [c1, c2, c3, c4])
    X_lr_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [c1, c2, c3, c4])

    # Initialize base learners
    base_xgb = XGBClassifier(**xgb_params)
    base_cat = CatBoostClassifier(**cat_params)
    base_lr = LogisticRegression(
        C=lr_C_base, solver=lr_solver_base,
        max_iter=1000,
        n_jobs=None if lr_solver_base == "liblinear" else -1
    )

    # Get out-of-fold predictions and test predictions
    oof_xgb, test_xgb = get_oof_preds(base_xgb, X_xgb_train, y_train_, X_xgb_test)
    oof_cat, test_cat = get_oof_preds(base_cat, X_cat_train, y_train_, X_cat_test)
    oof_lr, test_lr = get_oof_preds(base_lr, X_lr_train, y_train_, X_lr_test)

    # Stack predictions
    meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
    meta_test = np.hstack([test_xgb, test_cat, test_lr])

    # Train meta-learner
    meta_model = LogisticRegression(
        C=lr_C_meta,
        solver=lr_solver_meta,
        max_iter=1000,
        n_jobs=None if lr_solver_meta == "liblinear" else -1
    )
    meta_model.fit(meta_train, y_train_)

    # Predict probabilities and compute recall at threshold=0.3
    meta_probs = meta_model.predict_proba(meta_test)[:, 1]
    threshold = 0.3
    meta_preds = (meta_probs >= threshold).astype(int)
    recall = recall_score(y_test_, meta_preds, pos_label=1)
    return recall

# Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best recall:", study.best_value)
print("Best parameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Train final model with best parameters
best = study.best_params

# Reconstruct weighted feature sets with best weights
a1, a2, a3, a4 = best["xgb_w_effnet"], best["xgb_w_resnet"], best["xgb_w_vit"], best["xgb_w_tab"]
b1, b2, b3, b4 = best["cat_w_effnet"], best["cat_w_resnet"], best["cat_w_vit"], best["cat_w_tab"]
c1, c2, c3, c4 = best["lr_w_effnet"], best["lr_w_resnet"], best["lr_w_vit"], best["lr_w_tab"]

X_xgb_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [a1, a2, a3, a4])
X_xgb_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [a1, a2, a3, a4])

X_cat_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [b1, b2, b3, b4])
X_cat_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [b1, b2, b3, b4])

X_lr_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [c1, c2, c3, c4])
X_lr_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [c1, c2, c3, c4])

# Initialize final models with best parameters (adjust if optimized hyperparams are included)
final_xgb = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.7,
    reg_lambda=2e-3,
    random_state=42,
    eval_metric="logloss",
    use_label_encoder=False,
    n_jobs=-1,
)

final_cat = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=2e-3,
    l2_leaf_reg=3,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=False,
)

final_lr_base = LogisticRegression(
    C=best["lr_base_C"],
    solver=best["lr_base_solver"],
    max_iter=1000,
    n_jobs=None if best["lr_base_solver"] == "liblinear" else -1,
)

# Get OOF predictions again for full dataset
oof_xgb, test_xgb = get_oof_preds(final_xgb, X_xgb_train, y_train_, X_xgb_test)
oof_cat, test_cat = get_oof_preds(final_cat, X_cat_train, y_train_, X_cat_test)
oof_lr, test_lr = get_oof_preds(final_lr_base, X_lr_train, y_train_, X_lr_test)

meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
meta_test = np.hstack([test_xgb, test_cat, test_lr])

# Final meta learner
final_meta = LogisticRegression(
    C=best["lr_meta_C"],
    solver=best["lr_meta_solver"],
    max_iter=1000,
    n_jobs=None if best["lr_meta_solver"] == "liblinear" else -1,
)
final_meta.fit(meta_train, y_train_)

final_probs = final_meta.predict_proba(meta_test)[:, 1]
final_preds = (final_probs >= 0.3).astype(int)

print("\nFinal AUC:", roc_auc_score(y_test_, final_probs))
print(classification_report(y_test_, final_preds, digits=4))


[I 2025-08-16 07:46:21,201] A new study created in memory with name: no-name-923219ff-cf8f-4682-a391-f76e7a5ed81b
[I 2025-08-16 07:47:09,728] Trial 0 finished with value: 0.6016949152542372 and parameters: {'xgb_w_effnet': 3.926140141085941, 'xgb_w_resnet': 2.7285790624334183, 'xgb_w_vit': 2.746697398490894, 'xgb_w_tab': 2.359875852910637, 'cat_w_effnet': 2.7944499838754497, 'cat_w_resnet': 2.808473109664984, 'cat_w_vit': 2.241863009681797, 'cat_w_tab': 4.504605891044994, 'lr_w_effnet': 2.186859979249937, 'lr_w_resnet': 2.909168529014704, 'lr_w_vit': 3.4543542050629297, 'lr_w_tab': 4.6309386003629704, 'lr_base_C': 0.28182211872118657, 'lr_base_solver': 'lbfgs', 'lr_meta_C': 0.5212869178111094, 'lr_meta_solver': 'saga'}. Best is trial 0 with value: 0.6016949152542372.
[I 2025-08-16 07:49:07,013] Trial 1 finished with value: 0.6016949152542372 and parameters: {'xgb_w_effnet': 2.570548955404649, 'xgb_w_resnet': 1.214235038762013, 'xgb_w_vit': 2.6935393522142337, 'xgb_w_tab': 3.95407484518

KeyboardInterrupt: 

In [92]:
# Rebuild weighted feature sets using Optuna's best parameters
a1, a2, a3, a4 = 1.733, 1.932, 3.476, 4.229
b1, b2, b3, b4 = 1.690, 4.320, 1.623, 3.969
c1, c2, c3, c4 = 4.946, 1.646, 1.685, 3.453

X_xgb_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [a1, a2, a3, a4])
X_xgb_test  = weighted_concat([X_test_effnet,  X_test_resnet,  X_test_vit_im,  X_test_tab_scaled],  [a1, a2, a3, a4])
X_cat_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [b1, b2, b3, b4])
X_cat_test  = weighted_concat([X_test_effnet,  X_test_resnet,  X_test_vit_im,  X_test_tab_scaled],  [b1, b2, b3, b4])
X_lr_train  = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [c1, c2, c3, c4])
X_lr_test   = weighted_concat([X_test_effnet,  X_test_resnet,  X_test_vit_im,  X_test_tab_scaled],  [c1, c2, c3, c4])

# Initialize final models with best parameters (from study)
final_xgb = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.7,
    reg_lambda=2e-3,
    random_state=42,
    eval_metric="logloss",
    use_label_encoder=False,
    n_jobs=-1,
)
final_cat = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=2e-3,
    l2_leaf_reg=3,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=False,
)
final_lr_base = LogisticRegression(
    C=0.028378,
    solver='liblinear',
    max_iter=1000,
    n_jobs=None if 'liblinear' == "liblinear" else -1,
)

# Get OOF predictions for stacking
oof_xgb, test_xgb = get_oof_preds(final_xgb, X_xgb_train, y_train_, X_xgb_test)
oof_cat, test_cat = get_oof_preds(final_cat, X_cat_train, y_train_, X_cat_test)
oof_lr,  test_lr  = get_oof_preds(final_lr_base, X_lr_train, y_train_, X_lr_test)

meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
meta_test  = np.hstack([test_xgb, test_cat, test_lr])

# Final meta learner
final_meta = LogisticRegression(
    C=best["lr_meta_C"],
    solver=best["lr_meta_solver"],
    max_iter=1000,
    n_jobs=None if best["lr_meta_solver"] == "liblinear" else -1,
)
final_meta.fit(meta_train, y_train_)
final_probs = final_meta.predict_proba(meta_test)[:, 1]

# --- Find best threshold for recall or F1 ---
import numpy as np
from sklearn.metrics import recall_score, f1_score

thresholds = np.arange(0.05, 0.95, 0.01)
recalls = []
f1s = []

for t in thresholds:
    preds = (final_probs >= t).astype(int)
    recalls.append(recall_score(y_test_, preds, pos_label=1))
    f1s.append(f1_score(y_test_, preds, pos_label=1))

best_recall_idx = np.argmax(recalls)
best_f1_idx = np.argmax(f1s)
best_recall_threshold = thresholds[best_recall_idx]
best_f1_threshold = thresholds[best_f1_idx]

print(f'Best recall: {recalls[best_recall_idx]:.4f} at threshold={best_recall_threshold:.2f}')
print(f'Best F1: {f1s[best_f1_idx]:.4f} at threshold={best_f1_threshold:.2f}')

# Classification report at recall-optimal threshold:
optimal_preds = (final_probs >= best_recall_threshold).astype(int)
print("\nClassification report (recall-optimal threshold):")
print(classification_report(y_test_, optimal_preds, digits=4))

print(f"\nFinal AUC: {roc_auc_score(y_test_, final_probs):.4f}")

# You can also print the report at any threshold, e.g., 0.3:
preds_03 = (final_probs >= 0.3).astype(int)
print("\nClassification report (threshold=0.3):")
print(classification_report(y_test_, preds_03, digits=4))


Best recall: 0.7373 at threshold=0.05
Best F1: 0.6957 at threshold=0.22

Classification report (recall-optimal threshold):
              precision    recall  f1-score   support

         0.0     0.9590    0.9052    0.9314       802
         1.0     0.5337    0.7373    0.6192       118

    accuracy                         0.8837       920
   macro avg     0.7464    0.8213    0.7753       920
weighted avg     0.9045    0.8837    0.8913       920


Final AUC: 0.9204

Classification report (threshold=0.3):
              precision    recall  f1-score   support

         0.0     0.9496    0.9626    0.9560       802
         1.0     0.7196    0.6525    0.6844       118

    accuracy                         0.9228       920
   macro avg     0.8346    0.8076    0.8202       920
weighted avg     0.9201    0.9228    0.9212       920



In [93]:
print("Best recall:", study.best_value)
print("Best parameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Train final model with best parameters
best = study.best_params

# Reconstruct weighted feature sets with best weights
a1, a2, a3, a4 = best["xgb_w_effnet"], best["xgb_w_resnet"], best["xgb_w_vit"], best["xgb_w_tab"]
b1, b2, b3, b4 = best["cat_w_effnet"], best["cat_w_resnet"], best["cat_w_vit"], best["cat_w_tab"]
c1, c2, c3, c4 = best["lr_w_effnet"], best["lr_w_resnet"], best["lr_w_vit"], best["lr_w_tab"]

X_xgb_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [a1, a2, a3, a4])
X_xgb_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [a1, a2, a3, a4])

X_cat_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [b1, b2, b3, b4])
X_cat_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [b1, b2, b3, b4])

X_lr_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled], [c1, c2, c3, c4])
X_lr_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled], [c1, c2, c3, c4])

# Initialize final models with best parameters (adjust if optimized hyperparams are included)
final_xgb = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.7,
    reg_lambda=2e-3,
    random_state=42,
    eval_metric="logloss",
    use_label_encoder=False,
    n_jobs=-1,
)

final_cat = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=2e-3,
    l2_leaf_reg=3,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=False,
)

final_lr_base = LogisticRegression(
    C=best["lr_base_C"],
    solver=best["lr_base_solver"],
    max_iter=1000,
    n_jobs=None if best["lr_base_solver"] == "liblinear" else -1,
)

# Get OOF predictions again for full dataset
oof_xgb, test_xgb = get_oof_preds(final_xgb, X_xgb_train, y_train_, X_xgb_test)
oof_cat, test_cat = get_oof_preds(final_cat, X_cat_train, y_train_, X_cat_test)
oof_lr, test_lr = get_oof_preds(final_lr_base, X_lr_train, y_train_, X_lr_test)

meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
meta_test = np.hstack([test_xgb, test_cat, test_lr])

# Final meta learner
final_meta = LogisticRegression(
    C=best["lr_meta_C"],
    solver=best["lr_meta_solver"],
    max_iter=1000,
    n_jobs=None if best["lr_meta_solver"] == "liblinear" else -1,
)
final_meta.fit(meta_train, y_train_)

final_probs = final_meta.predict_proba(meta_test)[:, 1]
final_preds = (final_probs >= 0.3).astype(int)

print("\nFinal AUC:", roc_auc_score(y_test_, final_probs))
print(classification_report(y_test_, final_preds, digits=4))

Best recall: 1.0
Best parameters:
  xgb_w_effnet: 1.4888826533510902
  xgb_w_resnet: 2.165285934146347
  xgb_w_vit: 1.0466640567729575
  xgb_w_tab: 4.777546598473762
  cat_w_effnet: 1.0720426982785107
  cat_w_resnet: 4.7466110146072245
  cat_w_vit: 1.472749754075691
  cat_w_tab: 4.831277153944761
  lr_w_effnet: 4.795690791296725
  lr_w_resnet: 1.0902868320572001
  lr_w_vit: 1.6985034089390503
  lr_w_tab: 2.261680719595783
  lr_base_C: 9.539611095464293
  lr_base_solver: liblinear
  lr_meta_C: 0.001313628816632507
  lr_meta_solver: liblinear

Final AUC: 0.9086077179931528
              precision    recall  f1-score   support

         0.0     0.0000    0.0000    0.0000       802
         1.0     0.1283    1.0000    0.2274       118

    accuracy                         0.1283       920
   macro avg     0.0641    0.5000    0.1137       920
weighted avg     0.0165    0.1283    0.0292       920



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [94]:
from sklearn.metrics import classification_report, roc_auc_score

final_meta.fit(meta_train, y_train_)
final_probs = final_meta.predict_proba(meta_test)[:, 1]
final_preds = (final_probs >= 0.3).astype(int)  # Apply threshold

print("\nFinal AUC:", roc_auc_score(y_test_, final_probs))
print(classification_report(y_test_, final_preds, digits=4))



Final AUC: 0.9086077179931528
              precision    recall  f1-score   support

         0.0     0.0000    0.0000    0.0000       802
         1.0     0.1283    1.0000    0.2274       118

    accuracy                         0.1283       920
   macro avg     0.0641    0.5000    0.1137       920
weighted avg     0.0165    0.1283    0.0292       920



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [95]:
from sklearn.metrics import classification_report, roc_auc_score

# Set best parameters from Optuna Trial 14
best = {
    'xgb_w_effnet': 1.7331561744250479,
    'xgb_w_resnet': 1.9326823244907394,
    'xgb_w_vit': 3.4763263231937636,
    'xgb_w_tab': 4.2296121105090565,
    'cat_w_effnet': 1.6902828025432388,
    'cat_w_resnet': 4.320616099846135,
    'cat_w_vit': 1.6239350940173336,
    'cat_w_tab': 3.969865861577566,
    'lr_w_effnet': 4.946334671351695,
    'lr_w_resnet': 1.6460402042432802,
    'lr_w_vit': 1.6857050255902633,
    'lr_w_tab': 3.4538273378641593,
    'lr_base_C': 0.028378712154222645,
    'lr_base_solver': 'liblinear',
    'lr_meta_C': 0.003373775445909113,
    'lr_meta_solver': 'liblinear'
}

# Rebuild weighted feature arrays for base learners using weighted_concat function
X_xgb_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
                             [best['xgb_w_effnet'], best['xgb_w_resnet'], best['xgb_w_vit'], best['xgb_w_tab']])
X_xgb_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
                            [best['xgb_w_effnet'], best['xgb_w_resnet'], best['xgb_w_vit'], best['xgb_w_tab']])

X_cat_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
                              [best['cat_w_effnet'], best['cat_w_resnet'], best['cat_w_vit'], best['cat_w_tab']])
X_cat_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
                             [best['cat_w_effnet'], best['cat_w_resnet'], best['cat_w_vit'], best['cat_w_tab']])

X_lr_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
                            [best['lr_w_effnet'], best['lr_w_resnet'], best['lr_w_vit'], best['lr_w_tab']])
X_lr_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
                           [best['lr_w_effnet'], best['lr_w_resnet'], best['lr_w_vit'], best['lr_w_tab']])

# Define base learners with fixed params + weighted features
final_xgb = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.7,
    reg_lambda=2e-3,
    random_state=42,
    eval_metric="logloss",
    use_label_encoder=False,
    n_jobs=-1,
)
final_cat = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=2e-3,
    l2_leaf_reg=3,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=False,
)
final_lr_base = LogisticRegression(
    C=best['lr_base_C'],
    solver=best['lr_base_solver'],
    max_iter=1000,
    n_jobs=None if best['lr_base_solver'] == 'liblinear' else -1,
)

# Get OOF predictions for stacking meta features
oof_xgb, test_xgb = get_oof_preds(final_xgb, X_xgb_train, y_train_, X_xgb_test)
oof_cat, test_cat = get_oof_preds(final_cat, X_cat_train, y_train_, X_cat_test)
oof_lr, test_lr = get_oof_preds(final_lr_base, X_lr_train, y_train_, X_lr_test)

meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
meta_test = np.hstack([test_xgb, test_cat, test_lr])

# Final meta learner (logistic regression)
final_meta = LogisticRegression(
    C=best['lr_meta_C'],
    solver=best['lr_meta_solver'],
    max_iter=1000,
    n_jobs=None if best['lr_meta_solver'] == 'liblinear' else -1,
)
final_meta.fit(meta_train, y_train_)
final_probs = final_meta.predict_proba(meta_test)[:, 1]

# Threshold predictions at 0.3
final_preds = (final_probs >= 0.3).astype(int)

# Print metrics
print("\nFinal ROC AUC:", roc_auc_score(y_test_, final_probs))
print(classification_report(y_test_, final_preds, digits=4))



Final ROC AUC: 0.9094530622596052
              precision    recall  f1-score   support

         0.0     0.9663    0.8217    0.8881       802
         1.0     0.3992    0.8051    0.5337       118

    accuracy                         0.8196       920
   macro avg     0.6827    0.8134    0.7109       920
weighted avg     0.8935    0.8196    0.8427       920



In [96]:
from sklearn.metrics import classification_report, roc_auc_score

# Use Trial 16 best parameters
best = {
    'xgb_w_effnet': 3.0167451146989634,
    'xgb_w_resnet': 3.0902868622019093,
    'xgb_w_vit': 2.308297646094281,
    'xgb_w_tab': 1.0715412565755038,
    'cat_w_effnet': 1.4578907959615115,
    'cat_w_resnet': 4.406617509575396,
    'cat_w_vit': 1.5508970407079867,
    'cat_w_tab': 3.884231716374087,
    'lr_w_effnet': 4.435458614212536,
    'lr_w_resnet': 1.8619291448670328,
    'lr_w_vit': 1.5647945343821021,
    'lr_w_tab': 1.2731386945264886,
    'lr_base_C': 9.298804997195974,
    'lr_base_solver': 'liblinear',
    'lr_meta_C': 0.003733882200252675,
    'lr_meta_solver': 'liblinear'
}

# Rebuild weighted data
X_xgb_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
                             [best['xgb_w_effnet'], best['xgb_w_resnet'], best['xgb_w_vit'], best['xgb_w_tab']])
X_xgb_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
                            [best['xgb_w_effnet'], best['xgb_w_resnet'], best['xgb_w_vit'], best['xgb_w_tab']])

X_cat_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
                              [best['cat_w_effnet'], best['cat_w_resnet'], best['cat_w_vit'], best['cat_w_tab']])
X_cat_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
                             [best['cat_w_effnet'], best['cat_w_resnet'], best['cat_w_vit'], best['cat_w_tab']])

X_lr_train = weighted_concat([X_train_effnet, X_train_resnet, X_train_vit_im, X_train_tab_scaled],
                            [best['lr_w_effnet'], best['lr_w_resnet'], best['lr_w_vit'], best['lr_w_tab']])
X_lr_test = weighted_concat([X_test_effnet, X_test_resnet, X_test_vit_im, X_test_tab_scaled],
                           [best['lr_w_effnet'], best['lr_w_resnet'], best['lr_w_vit'], best['lr_w_tab']])

# Initialize final models with trial 16 parameters
final_xgb = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.7,
    reg_lambda=2e-3,
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False,
    n_jobs=-1,
)
final_cat = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=2e-3,
    l2_leaf_reg=3,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=False,
)
final_lr_base = LogisticRegression(
    C=best['lr_base_C'],
    solver=best['lr_base_solver'],
    max_iter=1000,
    n_jobs=None if best['lr_base_solver'] == 'liblinear' else -1,
)

# Get OOF preds for stacking
oof_xgb, test_xgb = get_oof_preds(final_xgb, X_xgb_train, y_train_, X_xgb_test)
oof_cat, test_cat = get_oof_preds(final_cat, X_cat_train, y_train_, X_cat_test)
oof_lr, test_lr = get_oof_preds(final_lr_base, X_lr_train, y_train_, X_lr_test)

meta_train = np.hstack([oof_xgb, oof_cat, oof_lr])
meta_test = np.hstack([test_xgb, test_cat, test_lr])

# Final meta learner setup and fit
final_meta = LogisticRegression(
    C=best['lr_meta_C'],
    solver=best['lr_meta_solver'],
    max_iter=1000,
    n_jobs=None if best['lr_meta_solver'] == 'liblinear' else -1,
)
final_meta.fit(meta_train, y_train_)

# Predict final probabilities and apply threshold 0.3
final_probs = final_meta.predict_proba(meta_test)[:, 1]
final_preds = (final_probs >= 0.3).astype(int)

# Evaluation output
print("\nFinal AUC:", roc_auc_score(y_test_, final_probs))
print(classification_report(y_test_, final_preds, digits=4))



Final AUC: 0.9099285684094848
              precision    recall  f1-score   support

         0.0     0.9644    0.8441    0.9003       802
         1.0     0.4266    0.7881    0.5536       118

    accuracy                         0.8370       920
   macro avg     0.6955    0.8161    0.7269       920
weighted avg     0.8954    0.8370    0.8558       920

